In [ ]:
import os
import pandas as pd
import regex as re
import nltk
import os
from string import punctuation
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
!pip install pymorphy2
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import pickle
import numpy as np
import json

nltk.download('stopwords')  
nltk.download('punkt')
russian_stemmer = nltk.stem.snowball.SnowballStemmer("russian")
russian_stopwords = stopwords.words('russian')

In [2]:
!unzip -q  "/content/drive/My Drive/PROGA/infosearch/friends-data.zip"

In [3]:
list_of_paths = []

for root, dirs, files in os.walk("/content/friends-data"):
    for name in files:
        path = str(root + '/' +  name)
        list_of_paths.append(path)
        #print(os.path.join(root, name))

len(list_of_paths)

165

In [4]:
columns = ['text', 'document'] 

df = pd.DataFrame(columns = columns)
for i, element in enumerate(list_of_paths):
  path = list_of_paths[i]
  document = re.findall(r'\dx\d\d', path)[0]
  with open(path, 'r') as f:  
    text = f.read() 
    df = df.append({'text': text, 'document': document}, ignore_index=True)


def remove_with_regex(text):
    regex = re.compile("\d+")
    words = [re.sub(regex, "", word)  for word in text.split()]
    return ' '.join(words)


def remove_stowords(text):
    text = [word for word in text.split() if word.lower() not in russian_stopwords]
    return ' '.join(text)


def remove_punctuation_and_make_lowercase(text):
    sentences = [(sentence.translate(str.maketrans('', '', punctuation))).lower() for sentence in text.split()]
    return ' '.join(sentences)


def lemmatization(text):
  tokens = word_tokenize(text)
  list_tokens = []
  for token in tokens:
    p = morph.parse(str(token))[0]
    list_tokens.append(p.normal_form)

  return ' '.join(list_tokens)

# def stemmatize(text):
#     text = [russian_stemmer.stem(word) for word in text.split()]
#     return ' '.join(text)

lemmatized_texts = pd.DataFrame(columns=['lemmatized_text'])
for text in list(df['text']):   
  lemmatized_text = lemmatization(remove_punctuation_and_make_lowercase(remove_stowords(remove_with_regex(text))))
  lemmatized_texts = lemmatized_texts.append({'lemmatized_text': lemmatized_text}, ignore_index=True)

df['lemmatized_text'] = list(lemmatized_texts['lemmatized_text'])
df = df.sort_values(by = ['document'])
df

,text,document,lemmatized_text
124,﻿Друзья. Как все началось\n\nДа нечего рассказ...,1x01,﻿друг начаться нечего рассказывать просто сотр...
122,"﻿Вы, мужики, не понимаете\n\nДля нас поцелуй т...",1x02,﻿вы мужик понимать поцелуй важный остальной да...
131,"﻿- Привет, ребята.\n- Привет, Фиби..\n\n- Как ...",1x03,﻿ привет ребята привет фиби прошлый очень хоро...
125,"﻿Так, теперь Фиби?\n\nЕсли бы я была всемогуще...",1x04,﻿так фиби всемогущий день пожелать мир весь ми...
130,"﻿Может, хватит? Здесь нет ничего особенного\n\...",1x05,﻿мочь хватить особенный особенный это просто ч...
...,...,...,...
27,"﻿Как вы думаете,\nкто из нас женится следующим...",7x20,﻿как думать жениться следующий наверное моника...
28,"﻿Ты понимаешь, что до свадьбы остались\nвсего ...",7x21,﻿ты понимать свадьба остаться четыре неделя че...
40,﻿- Привет.\n- Привет.\n\nЯ видел припаркованны...,7x22,﻿ привет привет видеть припарковать порша улиц...
30,"﻿Вы хоть понимаете, что сегодня мы\nв последни...",7x23,﻿вы понимать сегодня последний встречаться зде...


In [5]:
# делаем матрицу

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['lemmatized_text'])
matrix = X.toarray()

with open('data.pkl', 'wb') as f:
  pickle.dump(matrix, f)

matrix.shape

(165, 15464)

In [18]:
# делаем json

names = vectorizer.get_feature_names()
count_list  = X.toarray()
word_dict = {name:count_list[:,n].tolist() for n,name in enumerate(names)}

for k,v in word_dict.items():
  word_dict[k] = dict(zip(df.document, v))

with open('friends.json', 'w') as json_file:
  json.dump(word_dict, json_file)

С помощью обратного индекса в формате Term-Document посчитайте:

a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) кто из главных героев статистически самый популярный?

In [17]:
names = vectorizer.get_feature_names()
data_matrix = pd.DataFrame(X.toarray(), columns = names)
data_matrix

,after,again,ahh,all,and,are,au,aнгел,bay,behind,bodingtons,bodingtonsдааа,bonjour,by,careless,cat,cd,cdпроигрыватель,cheese,china,chuck,city,co,cамый,cочельник,cтарый,cтоп,dais,dance,daysailer,delivery,dickin,dicky,didnt,dings,discovery,done,doobies,duh,dьzenleme,...,ям,ямагучить,ямочка,ямс,ямснямнямс,ян,январь,янки,янкиз,янковский,японец,япония,японский,яркий,ярко,ярковата,ярлык,ярлычок,ярмарка,ярость,ясердцеросс,ясмина,ясно,ясность,ясный,ятебеееотдать,ято,яхта,ящерица,ящик,ящичек,ёй,ёкнуть,ёлка,ёлочный,ёпэрэсотэ,ёрл,ёрш,ёршик,ёще
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
161,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
162,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
163,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [19]:
# какое слово является самым частотным
data = data_matrix.sum(axis = 0, skipna = True)
data = data.sort_values(ascending=False)
data.index[0]

'это'

In [20]:
# какое самым редким
df =  pd.DataFrame({'word':data.index, 'count':data.values})
for x, y in zip(df['word'], df['count']):
  if y == 1:
    print(x)

Выходные данные были обрезаны до нескольких последних строк (5000).
ващий
завод
ввергнуть
введение
вбузздить
завотдел
вбрасывать
вбрасывание
вблизи
вбегать
загадочный
ватикан
загасить
загибать
вафельный
вафельница
вафелек
заглотить
заглушить
ватрушечка
ватный
задвигон
валетов
вверху
валентинчик
бэтмобиль
бэмбить
бэй
бычок
зажарка
зажгуть
бычков
бычар
зажигательный
бытовой
зажиматься
зажить
зазазастрелиться
былаба
бык
заиграться
заикаться
буц
бухтеть
заинтересованность
буфы
бюллетень
задымить
задурить
задок
задержка
валентинка
валент
ваксить
задира
ваксинщица
ваксинга
вакантный
вайр
вайп
бюст
задолжать
задохнуться
вайза
вазон
вазелин
задрожать
вади
задумывать
вагон
завладеть
вводить
весна
забиться
заадвокатиться
вернитека
забавляться
веритечто
забастовка
вересковый
вербальный
веранда
забеременнеть
забеспокоиться
венчать
вентиль
венка
забиваться
венис
венец
венесуэла
венгерский
забирита
вена
велоцираптор
жёстко
вернуська
жёнушка
жпизод
весминстерский
жительство
житие
житься
жополиз
верюи

In [21]:
# кто из главных героев статистически самый популярный?

list_names = ['моника', 'рейчел','чендлер','фиби','росс','джоуи']
d = {}
for x, y in zip(df['word'], df['count']):
  if x in list_names:
    d[x] = y

for k, v in d.items():
  if v == max(d.values()):
    print(k, v)

росс 1015


In [36]:
# какой набор слов есть во всех документах коллекции
new_list = list(df['word'])
for word in new_list:
  if any(data_matrix[word]==0) == False:
    print(word)

это
ты
мочь
нет
да
хотеть
что
знать
мой
быть
думать
просто
